# Chainerによるリカレントニューラルネットワーク

##本チュートリアルではchainerを利用してリカレントニューラルネットワークの実装を確認，学習および評価を行います．　環境としてはGoogle が提供する Google Colaboratory上でおこないます． GPU上で処理を行うため，colaboratoryの[ランタイム]->[ランタイムのタイプを変更]からハードウェアアクセラレータをGPUにしてください．

Goolge Colaboratory上にChainerをインストールします．

In [0]:
!curl https://colab.chainer.org/install | sh -
!pip install tqdm
!pip install ipywidgets

リカレントニューラルネットワークを利用して電力を予測します．入力には，現時刻の電力，気温，湿度，曜日，時間を与えます．曜日は月曜から日曜までの７日のうち該当する曜日を1，それ以外を0とするone-hotベクトルとします．時間も同様に24時間のうち該当する時間を1，それ以外を0とします．これらを組み合わせて34次元のデータを入力として，未来の電力を予測します．予測する未来はあらかじめ学習時に指定します．指定された時刻の電力を正解として学習します．

Chainerでニューラルネットワークを学習するために必要なモジュールや関数をインポートします．

In [0]:
import numpy as np
from chainer import cuda,optimizers
import chainer, chainer
from chainer import Chain, Variable
import chainer.functions as F
import chainer.links as L
from chainer.dataset import convert

from os import path
from tqdm import tqdm_notebook as tqdm

GPUが利用できるか確認します．

In [0]:
print('GPU availability:', chainer.cuda.available)
print('cuDNN availablility:', chainer.cuda.cudnn_enabled)

リカレントニューラルネットワークのモデルを定義します．ここでは，２層の中間層をもつネットワークとします．入力データに対して，１層目はLSTMで構成し，２層目は全結合層とします．


In [0]:
class LSTM(chainer.Chain):
    def __init__(self):
        super(LSTM, self).__init__(
            l1=L.LSTM(34, 136),
            l3=L.Linear(136, 1),
        )

    def __call__(self, x, t):
        h = F.sigmoid(self.l1(x))
        h = F.sigmoid(self.l3(h))

        self.loss = F.mean_squared_error(h, t)
        return self.loss, h

    def reset_state(self):
        self.l1.reset_state()




ネットワークモデルをmodelと定義し，最適化手法にモーメンタム付きの確率的勾配降下法を利用します．

In [0]:
  model = LSTM()
#  cuda.get_device(0).use()
#  model.to_gpu()
  optimizer = optimizers.MomentumSGD(lr=0.0001, momentum=0.9)
  optimizer.setup(model)

学習するための各種パラメータを定義します．ミニバッチサイズをbatchsize，推定する電力の最大値をmax_energy，リカレントニューラルネットワークで扱う時間幅をtime_window， 学習エポック数をepoch，電力を推定する時間をdelayとします．ここでは，delayを1０としており，10分後の電力を予測します．

In [0]:
batchsize = 20
max_energy=3000
time_window = 10
epoch = 10
delay = 10

次に学習データを読み込みます．

In [0]:
!wget http://vision.cs.chubu.ac.jp/CVTutorial/SRC/ML_Lecture/SOLAR/data.zip
!unzip  data.zip

In [0]:
!ls data/train

In [0]:
data  = np.load(path.join('data', 'train', 'BEMS_RNN_train_data.npy' ))
label = np.load(path.join('data', 'train', 'BEMS_RNN_train_labels.npy'))
x_train = np.asarray(data[ : int(-delay/5)])
y_train = np.asarray(label[int(delay/5): ])

data  = np.load(path.join('data', 'test', 'BEMS_RNN_test_data.npy'))
label = np.load(path.join('data', 'test', 'BEMS_RNN_test_labels.npy'))
x_test = np.asarray(data[ : int(-delay/5)])
y_test = np.asarray(label[int(delay/5) : ])

y_train = y_train.reshape(len(y_train), 1)
y_test = y_test.reshape(len(y_test), 1)


In [0]:
print(x_train.shape, y_train.shape)

In [0]:
def one_epoch(model, optimizer, data, label, epoch, train, batchsize, time_window, max_energy):
    model.train = train
    total_loss = 0
    accum_loss = 0
    test_loss = 0 

#    xp = cuda.cupy
    xp = np

    if train:
        for i in range(0, data.shape[0]-time_window, time_window):
            model.reset_state()
            for j in range(0, time_window):
                x = chainer.Variable(xp.reshape(data[i+j], (1, 1, 34)))
                t = chainer.Variable(xp.reshape(label[i+j], (1, 1)))

                loss, y = model(x, t)
                accum_loss += loss
                total_loss += loss.data

            optimizer.target.cleargrads()
            accum_loss.backward()
            accum_loss.unchain_backward()
            accum_loss = 0
            optimizer.update()
            if i % 1000 == 0:
                print ('{} iteration : {}'.format(i, total_loss /i))


        print ('{} epoch : {}'.format(epoch, total_loss / len(data)))

    else:
        test_loss = 0 
        out_file = open('test_{:02d}.txt'.format(epoch), 'w')
        for batch in zip(data, label):
            x = xp.asarray([batch[0]])
            t = xp.asarray([batch[1]]) 
            loss, y = model(x, t) 
            test_loss += loss.data
            out_file.write('{} {}\n'.format(int(t[0][0] *max_energy),int(y.data[0][0] *max_energy)))
        out_file.close()
        print ('{} epoch : {}'.format(epoch, test_loss / len(data)))


In [0]:
for ep in range(1, epoch + 1):
    print('epoch:{}\n'.format(ep))
    one_epoch( model, optimizer, x_train, y_train, ep, True, batchsize, time_window, max_energy)
    one_epoch( model, optimizer, x_test, y_test, ep, False, batchsize, time_window, max_energy)


In [0]:
!ls

In [0]:
!zip -r result-seq-b10.zip *.txt

In [0]:
from google.colab import files

files.download('result-seq-b10.zip')

In [0]:
model = LSTM()
cuda.get_device(0).use()
model.to_gpu()
optimizer = optimizers.MomentumSGD(lr=0.0001, momentum=0.9)
optimizer.setup(model)

In [0]:
def one_epoch(model, optimizer, data, label, epoch, train, batchsize, time_window, max_energy):
    model.train = train
    total_loss = 0
    accum_loss = 0
    test_loss = 0 

    xp = cuda.cupy
#    xp = np

    if train:
        for i in range(0, data.shape[0]-time_window, time_window):
            model.reset_state()
            for j in range(0, time_window):
                x = chainer.Variable(xp.asarray([data[i+j]]))
                t = chainer.Variable(xp.asarray([label[i+j]]))
#                x = chainer.Variable(xp.reshape(data[i+j], (1, 1, 34)))
#                t = chainer.Variable(xp.reshape(label[i+j], (1, 1)))

                loss, y = model(x, t)
                accum_loss += loss
                total_loss += loss.data

            optimizer.target.cleargrads()
            accum_loss.backward()
            accum_loss.unchain_backward()
            accum_loss = 0
            optimizer.update()
            if i % 1000 == 0:
                print ('{} iteration : {}'.format(i, total_loss /i))


        print ('{} epoch : {}'.format(epoch, total_loss / len(data)))

    else:
        test_loss = 0 
        out_file = open('test_{:02d}.txt'.format(epoch), 'w')
        for batch in zip(data, label):
            x = xp.asarray([batch[0]])
            t = xp.asarray([batch[1]]) 
            loss, y = model(x, t) 
            test_loss += loss.data
            out_file.write('{} {}\n'.format(int(t[0][0] *max_energy),int(y.data[0][0] *max_energy)))
        out_file.close()
        print ('{} epoch : {}'.format(epoch, test_loss / len(data)))


In [0]:
for ep in range(1, epoch + 1):
    print('epoch:{}\n'.format(ep))
    one_epoch( model, optimizer, x_train, y_train, ep, True, batchsize, time_window, max_energy)
    one_epoch( model, optimizer, x_test, y_test, ep, False, batchsize, time_window, max_energy)


In [0]:
model = LSTM()
cuda.get_device(0).use()
model.to_gpu()
optimizer = optimizers.MomentumSGD(lr=0.0001, momentum=0.9)
optimizer.setup(model)

In [0]:
def one_epoch(model, optimizer, data, label, epoch, train, batchsize, time_window, max_energy):
    model.train = train
    total_loss = 0
    accum_loss = 0
    test_loss = 0 

    xp = cuda.cupy
    
    if train:
        perm = np.random.permutation(data.shape[0] - time_window)
        print (' # of iterations : {}'.format(data.shape[0] / batchsize))
        for i in  range(0, len(perm), batchsize) :
            model.reset_state()
            start = perm[i:i+batchsize]
            for j in range(0, time_window):
                current = start + j                
                x = chainer.Variable(xp.asarray(data[current]))
                t = chainer.Variable(xp.asarray(label[current]))

                loss, y = optimizer.target(x, t)
                accum_loss += loss
                total_loss += loss.data

            optimizer.target.cleargrads()
            accum_loss.backward()
            accum_loss.unchain_backward()
            accum_loss = 0
            optimizer.update()
            if i/batchsize % 100 == 0:
                print ('{} iteration : {}'.format(i/batchsize, total_loss / (i * batchsize)))

        print ('{} epoch : {}'.format(epoch, total_loss / len(data)))

    else:
        test_loss = 0 
        out_file = open('test_{:02d}.txt'.format(epoch), 'w')
        for batch in zip(data, label):
            x = xp.asarray([batch[0]])
            t = xp.asarray([batch[1]]) 
            loss, y = model(x, t) 
            test_loss += loss.data
            out_file.write('{} {}\n'.format(int(t[0][0] *max_energy),int(y.data[0][0] *max_energy)))
        out_file.close()
        print ('{} epoch : {}'.format(epoch, test_loss / len(data)))

In [0]:
for ep in range(1, epoch + 1):
    print('epoch:{}\n'.format(ep))
    one_epoch( model, optimizer, x_train, y_train, ep, True, batchsize, time_window, max_energy)
    one_epoch( model, optimizer, x_test, y_test, ep, False, batchsize, time_window, max_energy)


In [0]:
from google.colab import files
!zip -r result-random-b10.zip ./*.txt
files.download('result-random-b10.zip')

## 課題　
###以下の課題に取り組みましょう
1  GPUに対応したコードにしましょう

2  学習データをランダムに利用するように修正しましょう

3  ミニバッチ学習できるようにしましょう

4 LSTMの代わりにGRUを利用してネットワークを作りましょう
